This code assumes that want to continuing the training of pre-defined model. In this example, we load not only the model (from .ckpt.meta file by the saver = tf.train.import_meta_graph function) but also the value of the model parameters (from .ckpt file by the saver.restore function). 

Alternatively, if you have the code to construct the model, then you just need to load the model parameters (from .ckpt file by the saver.restore function).

As long as both the model and the parameters are ready, we can continue the training. 

Note: here we can skip variable initialization (by tf.global_variables_initializer) since we use the saver.restore function.

Import tf and mnist data. Define X_train, y_train, X_val and y_val

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('mnist')

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [4]:
X_train, y_train = mnist.train.images, mnist.train.labels

In [5]:
X_val, y_val = mnist.validation.images, mnist.validation.labels

Import the model nodes from tmp/mnist_BN into the defualt graph

In [6]:
saver = tf.train.import_meta_graph('tmp/mnist_BN.ckpt.meta')

List the names of all operations/tensors of the restored graph. List the names of all nodes to observe.

In [7]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
training/input
training
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
batch_norm1/gamma/Initializer/ones
batch_norm1/gamma
batch_norm1/gamma/Assign
batch_norm1/gamma/read
batch_norm1/beta/Initializer/zeros
batch_norm1/beta
batch_norm1/beta/Assign
batch_norm1/beta/read
batch_norm1/moving_mean/Initializer/zeros
batch_norm1/moving_mean
batch_norm1/moving_mean/Assign
batch_norm1/moving_mean/read
batch_norm1/moving_variance/Initializer/ones
batch_norm1/moving_variance
batch_norm1/moving_variance/Assign
batch_norm1/moving_variance/rea

Obtain tensors/operations needed to re-training

In [8]:
X = tf.get_default_graph().get_tensor_by_name('X:0')

In [9]:
X

<tf.Tensor 'X:0' shape=(?, 784) dtype=float32>

In [10]:
y = tf.get_default_graph().get_tensor_by_name('y:0')

In [11]:
y

<tf.Tensor 'y:0' shape=<unknown> dtype=int32>

In [12]:
training_op = tf.get_default_graph().get_operation_by_name('train/training_op')

In [13]:
training_op

<tf.Operation 'train/training_op' type=NoOp>

In [14]:
training = tf.get_default_graph().get_tensor_by_name('training:0')

In [15]:
training

<tf.Tensor 'training:0' shape=() dtype=bool>

In [16]:
accuracy = tf.get_default_graph().get_tensor_by_name('eval/accuracy:0')

In [17]:
accuracy

<tf.Tensor 'eval/accuracy:0' shape=() dtype=float32>

Get other update ops, since this model uses batch normalization

In [18]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

Restore the model's status from tmp/mnist_BN.ckpt. And continue training the model for 10 more epoch with batch_size of 50. Print out the validation accuracy after each epoch.

In [19]:
n_epoch = 10
batch_size = 50
n_step = mnist.train.num_examples // batch_size

In [20]:
with tf.Session() as sess:
    saver.restore(sess, 'tmp/mnist_BN.ckpt')
    for epoch in range(n_epoch):
        for step in range(n_step):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops, accuracy], feed_dict={X:X_batch, y:y_batch, training:True})
        accuracy_eval = sess.run(accuracy, feed_dict={X:X_val, y:y_val, training:False})
        print('Epoch: {}, Validation accurcy: {:.4f}'.format(epoch, accuracy_eval))

INFO:tensorflow:Restoring parameters from tmp/mnist_BN.ckpt
Epoch: 0, Validation accurcy: 0.9840
Epoch: 1, Validation accurcy: 0.9818
Epoch: 2, Validation accurcy: 0.9812
Epoch: 3, Validation accurcy: 0.9850
Epoch: 4, Validation accurcy: 0.9852
Epoch: 5, Validation accurcy: 0.9844
Epoch: 6, Validation accurcy: 0.9848
Epoch: 7, Validation accurcy: 0.9844
Epoch: 8, Validation accurcy: 0.9850
Epoch: 9, Validation accurcy: 0.9846
